# Proyek Analisis Data: Brazilian E-Commerce Public Dataset by Olist
- **Nama:** Albar Pambagio Arioseto
- **Email:** albarpambagio@gmail.com
- **ID Dicoding:** albarpambagio 

## Menentukan Pertanyaan Bisnis

- 5 kota dengan jumlah order terbanyak?
- 5 kota dengan nilai transaksi tertinggi?

## Import Semua Packages/Library yang Digunakan

In [1]:
import pandas as pd
import plotly.express as px

## Data Wrangling

### Gathering Data

In [ ]:
df_ct = pd.read_csv('data\olist_customers_dataset.csv')
df_pm = pd.read_csv('data\olist_order_payments_dataset.csv')
df_od = pd.read_csv('data\olist_orders_dataset.csv')

In [24]:
# Export to Excel with 3 sheets
with pd.ExcelWriter("brazilian_ecommerce_data.xlsx") as writer:
    df_ct.to_excel(writer, sheet_name="customers", index=False)
    df_pm.to_excel(writer, sheet_name="payments", index=False)
    df_od.to_excel(writer, sheet_name="orders", index=False)

**Insight:**
- dataset bersumber dari kaggle
- format dataset: csv

### Assessing Data

#### Checking missing value

In [3]:
df_ct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [4]:
df_pm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [5]:
df_od.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


#### Checking duplicates

In [6]:
df_ct.duplicated().sum()

0

In [7]:
df_pm.duplicated().sum()

0

In [8]:
df_od.duplicated().sum()

0

#### Checking outlier

In [9]:
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    
    return outliers

In [10]:
df_pm.columns

Index(['order_id', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value'],
      dtype='object')

In [11]:
outliers = detect_outliers_iqr(df_pm, 'payment_value')
outliers

,order_id,payment_sequential,payment_type,payment_installments,payment_value
23,4d680edbaa7d3d9bed69532957368a03,1,credit_card,10,353.09
26,d0a945f85ba1074b60aac97ade7e240e,1,credit_card,2,541.00
52,2e2c60b99754ae1e4d8b18846cfec9f2,1,credit_card,4,542.66
54,95442deb81a5d91c97c0df96b431634a,1,boleto,1,368.98
61,3ab4eea6776259ae8dfbabaeffece352,1,credit_card,8,387.80
...,...,...,...,...,...
103814,df8ab61cb615d551483618533aceb364,1,credit_card,10,713.65
103843,262bf694d9c87de7844dd80b98fcfef3,1,credit_card,5,372.08
103861,691688405bcee3ff9023639514498962,1,credit_card,1,356.53
103881,0406037ad97740d563a178ecc7a2075c,1,boleto,1,363.31


In [12]:
df_pm['payment_value'].describe()

count    103886.000000
mean        154.100380
std         217.494064
min           0.000000
25%          56.790000
50%         100.000000
75%         171.837500
max       13664.080000
Name: payment_value, dtype: float64

In [13]:
fig = px.box(df_pm, y='payment_value', title="Boxplot for payment value")
fig.show()

In [14]:
# Merge df_od with df_ct on the common column (e.g., 'customer_id')
merged_df = df_od.merge(df_ct[['customer_city', 'customer_id']], on='customer_id', how='left')

# Group by 'customer_city' and count 'order_delivered_customer_date' in merged_df
count_orders = merged_df.groupby('customer_city')['order_delivered_customer_date'].count().reset_index()

count_orders.rename(columns={'order_delivered_customer_date': 'order_count'}, inplace=True)

In [15]:
count_orders.shape

(4119, 2)

In [16]:
detect_outliers_iqr(count_orders, 'order_count')

,customer_city,order_count
23,adamantina,35
50,aguas de lindoia,22
55,aguas lindas de goias,24
70,alagoinhas,40
76,alegrete,35
...,...,...
4088,vitoria da conquista,88
4097,volta redonda,227
4098,votorantim,84
4099,votuporanga,63


In [17]:
count_orders['order_count'].describe()

count     4119.000000
mean        23.422190
std        270.110405
min          0.000000
25%          1.000000
50%          3.000000
75%          8.000000
max      15046.000000
Name: order_count, dtype: float64

In [18]:
fig_o = px.box(count_orders, y='order_count', title="Boxplot for order_count")
fig_o.show()

**Insight:**
- tidak terdapat missing value
- tidak terdapat duplicate data
- ada sekitar 7% outliers di kolom payment value
- ada sekitar 13% outliers di kolom order count

### Cleaning Data

**Insight:**
- Outlier tidak ditangani secara khusus karena diperlukan investigasi lebih lanjut untuk memastikan apakah outlier tersebut merupakan kesalahan data atau justru mencerminkan kondisi sebenarnya.
- Selain itu, tidak dilakukan proses penghapusan (dropping) maupun imputasi data, karena hal tersebut tidak dibutuhkan untuk menjawab dua pertanyaan analisis yang telah disebutkan sebelumnya.

## Exploratory Data Analysis (EDA)

### Explore ...

In [19]:
# Show top 5 rows from df_ct based on 'order_count'
top_5_order_count = count_orders.nlargest(5, 'order_count')
top_5_order_count

,customer_city,order_count
3597,sao paulo,15046
3155,rio de janeiro,6604
453,belo horizonte,2697
558,brasilia,2071
1143,curitiba,1489


In [20]:
# Step 1: Merge df_od and df_ct on 'customer_id' to add 'customer_city' to df_od
merged_df = df_od.merge(df_ct[['customer_id', 'customer_city']], on='customer_id', how='left')

# Step 2: Merge the result with df_pm on 'order_id' to add 'payment_value'
merged_df = merged_df.merge(df_pm[['order_id', 'payment_value']], on='order_id', how='left')

# Step 3: Group by 'customer_city' and calculate the sum of 'payment_value'
payment_by_city = merged_df.groupby('customer_city')['payment_value'].sum().reset_index()

In [21]:
# Show top 5 rows from payment_by_city based on 'payment_value'
top_5_payment_by_city = payment_by_city.nlargest(5, 'payment_value')
top_5_payment_by_city

,customer_city,payment_value
3597,sao paulo,2203373.09
3155,rio de janeiro,1161927.36
453,belo horizonte,421765.12
558,brasilia,354216.78
1143,curitiba,247392.48


## Visualization & Explanatory Analysis

### Pertanyaan 1:

In [22]:
# Visualize Top 5 from df_ctn based on 'order_count'
fig_ctn = px.bar(top_5_order_count, 
                 x='customer_city', 
                 y='order_count', 
                 title="Top 5 Cities by Order Count", 
                 labels={'order_count': 'Order Count', 'customer_city': 'Customer City'},
                 color='customer_city')

# Show the first chart (Order Count)
fig_ctn.show()

### Pertanyaan 2:

In [23]:
# Visualize Top 5 from payment_by_city based on 'payment_value'
fig_payment = px.bar(top_5_payment_by_city, 
                     x='customer_city', 
                     y='payment_value', 
                     title="Top 5 Cities by Payment Value", 
                     labels={'payment_value': 'Payment Value', 'customer_city': 'Customer City'},
                     color='customer_city')

# Show the second chart (Payment Value)
fig_payment.show()

## Conclusion

- Sejauh ini, perhitungan jumlah pesanan (order count) dan nilai pembayaran (payment value) menunjukkan konsistensi, di mana kota-kota dengan jumlah pesanan tinggi juga berkontribusi signifikan terhadap nilai transaksi yang besar.